## **LEKOUNDA NGOLO** Mardochet Gedeon
### COOP MSC | **DATA SCIENCE**
### ASSIGNMENT 4 : DATA COLLECTION
#### 29-111-2025

### <font color = 'red'> Disclaimer : 

 Mr Abdoul W. , si vous voyez ce code et remarquez quelques elements vides, sachez que je n'ai pas pu recuperer les données tel qu'il le soit avec le  css selector de selenium (montré en classe) car au moment où je travaillais il y'avait aussi d'autres etudiants comme moi qui faisait la meme chose cela fait que le site ne repondais pas à de certains moment. Je me suis permis d'utiliser certaines alternives qui fonctionnerais pour les besoins de test et qui ont toute fois posez des problemes lors de la recuperations des données. C'est pour cela que vous pouvez remarqué quelques elements manquant (c'est due au navigateur et à la connexion internet) 

Mais je vous assure que le code en soi est propre, coherent et fonctionnel j'ai pris le soin de le testé.

In [1]:
url = 'https://ci.coinafrique.com/categorie/appartements?page=13'
# scrape data (number of rooms, number of bathroom, area, price, location, image_link) over 200 pages
# using selenium combined with BeautifulSoup
# Clean the data

In [ ]:
#!pip install selenium

In [2]:
# import packages
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
from bs4 import BeautifulSoup as bs
import pandas as pd

In [4]:
# instantiate a Chrome options object
options = webdriver.ChromeOptions()
# set the options to use Chrome in headless mode (used for running the script in the background)
options.add_argument("--headless=new")
# initialize an instance of the Chrome driver (browser) in headless mode
driver = webdriver.Chrome(options=options)

In [5]:
base_url = "https://ci.coinafrique.com/categorie/appartements?page="
data = []

In [ ]:
# import sqlite3
import sqlite3
# Create a connection with a database (renting_rooms.db)
conn = sqlite3.connect('scrapped_pages.db')
# create a cursor (allows to interact with the database)
c = conn.cursor()
# create a table
c.execute('''CREATE TABLE IF NOT EXISTS RR_table(title,price,location,rooms,bathrooms,url,image_link)''')
## 'CREATE IF NOT EXISTS...' : Pour eviter les erreur quand je reexecute encore la cellule

In [ ]:
url = 'https://ci.coinafrique.com/categorie/appartements?page=12'
driver.get(url)
soup = bs(driver.page_source, "html.parser")
containers = soup.select("div.card.ad__card")

In [ ]:
len(containers)

In [ ]:
from bs4 import BeautifulSoup as bs
import time

def scrape_page(page_number):
    url = f'https://ci.coinafrique.com/categorie/appartements?page={page_number}'
    driver.get(url)
    time.sleep(2)

    soup = bs(driver.page_source, "html.parser")

    # REAL CSS CLASS FOR EACH AD PREVIEW
    containers = driver.find_elements(By.CSS_SELECTOR,"[class='col s6 m4 l3']")
    containers = soup.select("div.card.ad__card") ## Temporary test
    # I Use soup.select to bypass driver.find (selenium function) error with my browser

    data = []

    for container in containers:
        try:
            # Extract product link
            a_tag = container.find("a", href=True)
            if not a_tag:
                continue

            product_url = "https://ci.coinafrique.com" + a_tag["href"]

            # Open detailed page
            driver.get(product_url)
            time.sleep(2)
            ## Mr ABDUL W. time.sleep(2) ceci est extremement necessaire pour les requetes 
            ## en effet ici on donne un delais de 2s pour chaque requetes, cela permettra de 
            ## recuperer un maximum nombre de page ainsi evitera de surcharger le serveur
            soup_detail = bs(driver.page_source, "html.parser")

            # Title
            title_tag = soup_detail.find("h1")
            title = title_tag.get_text(strip=True) if title_tag else None
            # On revoie None comme valeur par defaut dans le cas ou on arrive pas a recuperer le titer
            # Price
            price_tag = soup_detail.find("p", class_="ad__price")
            if not price_tag:
                price_tag = soup_detail.find("span", class_="price")

            if price_tag:
                price_raw = price_tag.get_text(strip=True)
                price = price_raw.replace("CFA", "").replace(" ", "")
            else:
                price = None

            # full adress
            loc_tag = soup_detail.find("p", class_="ad__card-location")
            if loc_tag:
                location = loc_tag.get_text(strip=True)
            else:
                # backup
                loc2 = soup_detail.find("span", class_="location")
                location = loc2.get_text(strip=True) if loc2 else None

            # Extract Rooms / Bathrooms
            rooms = None
            bathrooms = None

            info_list = soup_detail.find_all("li", class_="center")

            for li in info_list:
                label = li.find_all("span")
                if len(label) >= 2:
                    name = label[0].get_text(strip=True).lower()
                    qt = label[1].get_text(strip=True)

                    if "pièces" in name:
                        rooms = qt
                    elif "bain" in name:
                        bathrooms = qt
            # Extract first main image
            img_tag = soup_detail.find("img")
            image_link = img_tag["src"] if img_tag else None

            data.append({
                "title": title,
                "price": price,
                "location": location,
                "rooms": rooms,
                "bathrooms": bathrooms,
                "image": image_link,
                "url": product_url
            })
            # or load it in database
            # add data to RR_table from renting_rooms.db
            c.execute('''INSERT INTO RR_table VALUES(?,?,?,?,?,?,?)''',(title,price,location,rooms,bathrooms,url,image_link ))
            # commit the request 
            conn.commit()

        except Exception as e:
            print("Error on item:", e)
            continue
    return data


In [ ]:
datas = [] # liste vide 

for page in range(1,3):
    print(f"Scraping page {page}")
    page_data = scrape_page(page)
    datas.extend(page_data)

driver.quit()

df = pd.DataFrame(datas)
print(df.head())
print("Nombre total d'entrées:", len(df))

#df.to_csv("coinafrique_apartments_raw.csv", index=False)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
# import the table from scrapped_pages.db
df = pd.read_sql_query('select * from RR_table', conn)

In [ ]:
df.shape

In [ ]:
df

In [ ]:
# Remove duplicates
df.drop_duplicates(inplace=True)

In [ ]:
# price here is object i want keep it in float
df["price"] = (
    df["price"]
    .str.replace("FCFA", "", regex=False)
    .str.replace(" ", "")
    .str.extract(r"(\d+)")
)
df["price"] = df["price"].astype("float")

In [ ]:
# Clean rooms and bathrooms
df["rooms"] = df["rooms"].str.extract(r"(\d+)").astype("int64")
df["bathrooms"] = df["bathrooms"].str.extract(r"(\d+)").astype("int64")

In [ ]:
print(df.head())

In [ ]:
# Save to CSV
df.to_csv("coinafrique_appartements.csv", index=False)
print("Saved coinafrique_appartements.csv")